In [1]:
import torch, requests, math
from io import BytesIO
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration, BitsAndBytesConfig

c:\Users\namth\anaconda3\envs\lung_caption_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ----------------------------
# 0) เตรียมอุปกรณ์
# ----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

device: cuda


In [ ]:
# ----------------------------
# 1) โหลดโมเดล + โปรเซสเซอร์
# ----------------------------
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    trust_remote_code=True,
)

bnb_cfg = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    dtype="auto",         # ถ้าใช้ torch>=2.4 แนะนำ dtype=torch.float16/bfloat16
    device_map="auto",
    trust_remote_code=True,
    offload_folder="offload",   # บางพารามิเตอร์ย้ายไป disk
    quantization_config=bnb_cfg,
).eval()

In [4]:
# ----------------------------
# 2) โหลดภาพทดสอบ + คำสั่ง
# ----------------------------
url = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
image = Image.open(BytesIO(requests.get(url, timeout=15).content)).convert("RGB").resize((224, 224))

messages = [{
    "role": "user",
    "content": [
        {"type": "image", "image": image},
        {"type": "text", "text": "Describe this image."}
    ],
}]

In [5]:
# ----------------------------
# 3) Inference ให้เห็นผลข้อความจริง
# ----------------------------
chat_text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# สร้าง batch inputs (ทั้งข้อความ+ภาพ) สำหรับ generate
inputs = processor(
    text=[chat_text],
    images=[image],
    videos=None,
    padding=True,
    return_tensors="pt",
)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

with torch.inference_mode():
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=16,
        do_sample=False,        # deterministic
        use_cache=True,
    )

# ตัด prompt ออก เหลือเฉพาะที่โมเดล "พูดต่อ"
trimmed = [out[len(inp):] for inp, out in zip(inputs["input_ids"], gen_ids)]
texts = processor.batch_decode(trimmed, skip_special_tokens=True)
print("Model output:", texts[0].strip())

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model output: The image depicts a serene beach scene during what appears to be either sunrise or sunset


In [ ]:
# ----------------------------
# 4) ดึง Visual Attention จาก Vision Tower (ViT)
#    - ใช้พิกเซลจาก image_processor โดยตรง
#    - ขอ output_attentions=True เพื่อดึงแผนที่ attention ทุกเลเยอร์
# ----------------------------

print(model.config.to_dict().keys())
print(model.config.output_attentions)   # ค่า default (True/False)

dict_keys(['vision_config', 'text_config', 'image_token_id', 'video_token_id', 'return_dict', 'output_hidden_states', 'torchscript', 'dtype', 'pruned_heads', 'tie_word_embeddings', 'chunk_size_feed_forward', 'is_encoder_decoder', 'is_decoder', 'cross_attention_hidden_size', 'add_cross_attention', 'tie_encoder_decoder', 'architectures', 'finetuning_task', 'id2label', 'label2id', 'task_specific_params', 'problem_type', 'tokenizer_class', 'prefix', 'bos_token_id', 'pad_token_id', 'eos_token_id', 'sep_token_id', 'decoder_start_token_id', 'max_length', 'min_length', 'do_sample', 'early_stopping', 'num_beams', 'temperature', 'top_k', 'top_p', 'typical_p', 'repetition_penalty', 'length_penalty', 'no_repeat_ngram_size', 'encoder_no_repeat_ngram_size', 'bad_words_ids', 'num_return_sequences', 'output_scores', 'return_dict_in_generate', 'forced_bos_token_id', 'forced_eos_token_id', 'remove_invalid_values', 'exponential_decay_length_penalty', 'suppress_tokens', 'begin_suppress_tokens', 'num_beam_

In [ ]:
with torch.inference_mode():
    out = model(
        **inputs,
        output_attentions=True,
        return_dict=True,
        use_cache=False
    )

print(type(out.attentions))

<class 'tuple'>


In [ ]:
print(len(out.attentions))      # จำนวนเลเยอร์
print(out.attentions)           # เลเยอร์สุดท้าย

36
(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)



🔎 ปัญหาของรุ่น Instruct

* **Qwen2.5-VL-3B-Instruct** เป็นรุ่นที่ fine-tune เพื่อสนทนา (chat) และตอบคำถาม
* ทีมผู้พัฒนาปิด `output_attentions` → เวลาขอ attention จะได้ `None`
* hook ก็ยังไม่เจอค่า เพราะ attention weights ถูกทิ้งไปใน forward

✅ วิธีแก้
* ต้องเปลี่ยนไปใช้ base แทน instruct "Qwen/Qwen2.5-VL-3B"
* แต่ผู้พัฒนาดันลบโมเดล base ไปจาก huggingface แล้ว → พอไปตามดูใน github ถึงจะใช้ชื่อ Qwen2.5-VL แต่เนื้อในกลายเป็น Instruct ไปแล้ว T_T

In [ ]:
# # ----------------------------
# # 5) รีเชปเป็นกริดแพตช์ แล้วอัพสเกลทับภาพ
# #    - คำนวณขนาดกริดจาก image_processor
# #    - สำหรับ ViT base (patch=14x14 ที่ 224), แต่เราอ่านจากพิกเซลจริงปลอดภัยกว่า
# # ----------------------------
# # หา H,W หลัง preprocess แล้วคำนวณจำนวนแพตช์โดยดูจาก config ของ vision_model
# # ถ้า vision_model.config.patch_size มีอยู่ ให้ใช้
# if hasattr(vision_model.config, "image_size"):
#     img_size_proc = vision_model.config.image_size  # ปกติ 224 หรือ 448 (ขึ้นกับ processor)
# else:
#     # สำรอง: ใช้ขนาดของ pixel_values
#     _, _, Hpx, Wpx = vision_inputs["pixel_values"].shape
#     img_size_proc = max(Hpx, Wpx)

# patch = getattr(vision_model.config, "patch_size", 14)  # สำรองค่า 14 ถ้าไม่มีระบุ
# Hp = math.ceil(img_size_proc / patch)
# Wp = math.ceil(img_size_proc / patch)

# # ตรวจว่าจำนวนโทเค็นตรงกับกริดไหม ไม่ตรงก็ปรับแบบยืดหยุ่น
# if attn_mean.numel() != Hp * Wp:
#     # บางครั้งมี token ไม่ครบเต็มกริดเพราะ padding; ลองเดาว่าจำนวนจริงคือ (N-1)
#     # แล้วใส่ค่า sqrt ลงตัวเพื่อหากริดสี่เหลี่ยมที่ใกล้เคียงที่สุด
#     Np = attn_mean.numel()
#     s = int(round(math.sqrt(Np)))
#     Hp = s
#     Wp = Np // s

# fmap = attn_mean.reshape(Hp, Wp).numpy()
# fmap = (fmap - fmap.min()) / (fmap.max() - fmap.min() + 1e-6)

# # อัพสเกล heatmap ให้เท่าขนาดภาพต้นฉบับ
# img_w, img_h = image.size
# fmap_img = np.array(Image.fromarray((fmap * 255).astype(np.uint8)).resize((img_w, img_h), Image.BILINEAR)) / 255.0

In [ ]:
# # ----------------------------
# # 6) วาด heatmap ซ้อนทับ + บันทึกลงเครื่อง
# # ----------------------------
# plt.figure(figsize=(8, 6))
# plt.imshow(image)
# plt.imshow(fmap_img, alpha=0.5, cmap="jet")  # ซ้อนทับ
# plt.axis("off")
# plt.tight_layout()
# out_path = "attention_overlay.png"
# plt.savefig(out_path, dpi=150)
# plt.close()
# print(f"Saved visual attention overlay to: {out_path}")